# 1. Configuración de entorno

En esta sección validamos que nuestro entorno de trabajo esté correctamente configurado antes de comenzar el análisis.  
Los pasos incluyen:

1. **Versión de Python**  
   - Se verifica que esté instalada la versión **3.11 o superior** (se recomienda 3.13).  
   - Esto garantiza compatibilidad con librerías modernas de análisis de datos y machine learning.

2. **Importación de librerías base**  
   - Se cargan librerías fundamentales:  
     - `numpy`, `pandas`: manipulación y análisis de datos.  
     - `matplotlib`, `seaborn`: visualización de datos.  
     - `scipy`: funciones estadísticas.  
   - Además se configuran estilos gráficos y opciones de visualización en pandas para trabajar con tablas más grandes.

3. **Verificación de versiones críticas**  
   - Se comprueba que `scikit-learn` esté instalado y en una versión **>= 1.0.1**.  
   - Esto es esencial ya que `scikit-learn` se usará para el modelado (baseline y posteriores).

Con esta configuración inicial aseguramos que el entorno sea reproducible y que todas las dependencias necesarias estén listas antes de continuar con el **EDA** y el **baseline**.


In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

assert sys.version_info >= (3, 11), "Este notebook trabajo con python 3.11 o superiores (recomendado 3.13)"

print(f"Python {sys.version_info.major}.{sys.version_info.minor} instalado correctamente")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
sns.set_palette("husl")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Librerías importadas exitosamente")

In [ ]:
# Verificar versiones de librerías críticas
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1"), "Requiere scikit-learn >= 1.0.1"
print(f"scikit-learn {sklearn.__version__} instalado")

# 2. Metodología CRISP-DM
## 2.1. Comprensión del Negocio
El problema de Google Play Store  

**Contexto:**  
Es 2025. El mercado de aplicaciones móviles es altamente competitivo: millones de apps conviven en Google Play Store.  
Los desarrolladores buscan mejorar la visibilidad de sus aplicaciones y los usuarios dependen del **rating promedio** para decidir qué descargar.  

**Problema actual:**  
- El rating se conoce **solo después** de que los usuarios descargan y reseñan.  
- Las valoraciones son **altamente variables** y pueden depender de múltiples factores (categoría, descargas, precio, tamaño, tipo de app).  
- Los desarrolladores carecen de una herramienta para **estimar la calificación potencial** de una app antes o durante su lanzamiento.  
- La competencia es muy alta: una diferencia de décimas en rating puede significar miles de descargas menos.  

---

### 2.1.1. Solución propuesta  
Construir un **sistema automático de predicción de rating** de apps a partir de sus características disponibles en el dataset de Google Play Store.  

---

### 2.1.2. Definiendo el éxito  

**Métrica de negocio:**  
- Ayudar a los desarrolladores a anticipar la valoración probable de su app.  
- Reducir la dependencia de pruebas de mercado costosas o lentas.  
- Identificar características clave que favorecen una alta valoración (≥ 4.3).  

**Métrica técnica:**  
- Lograr un **Error Absoluto Medio (MAE) < 0.5 estrellas** en la predicción de rating.  
- Para la versión de clasificación (alta vs. baja calificación): obtener un **F1-score > 0.70**.  

**¿Por qué estos valores?**  
- El rating va de 1 a 5 → un error de 0.5 equivale a 10% de la escala.  
- Una diferencia de medio punto puede marcar la visibilidad de la app en el ranking.  
- Tasadores humanos (usuarios) también muestran variabilidad similar en sus calificaciones.  

---

### 2.1.3 Preguntas críticas antes de empezar  

1. **¿Realmente necesitamos ML?**  
   - Alternativa 1: Calcular el promedio de ratings por categoría → demasiado simple, no captura variabilidad.  
   - Alternativa 2: Reglas heurísticas (ej. “si es gratis y tiene muchas descargas, tendrá rating alto”) → insuficiente.  
   - **Conclusión:** Sí, ML es apropiado para capturar relaciones no lineales y múltiples factores.  

2. **¿Qué pasa si el modelo falla?**  
   - Transparencia: aclarar que es una estimación automática.  
   - Complementar con rangos de predicción (ej: intervalo de confianza).  
   - Mantener como referencia comparativa, no como único criterio de éxito.  

3. **¿Cómo mediremos el impacto?**  
   - Capacidad de anticipar apps con alta probabilidad de éxito.  
   - Ahorro de tiempo en validaciones preliminares.  
   - Insights para desarrolladores sobre qué factores influyen más en el rating.  

---


## 2.2. Comprensión de los Datos  

El objetivo de esta fase es explorar y entender el dataset de Google Play Store antes de construir modelos **(análisis exploratorio)**.  
Nos centraremos en:  

1. **Vista rápida del dataset**  
   - Identificar dimensiones (filas × columnas).  
   - Tipos de datos (numéricos, categóricos, texto, fechas).  
   - Valores faltantes obvios y rangos sospechosos.

2. **Descripción de variables**  
   - Revisar cada columna y entender su significado.  
   - Detectar qué variables podrían ser útiles como predictores y cuál será la variable objetivo (rating).  

3. **Detección de problemas en los datos**  
   - Análisis de valores faltantes.  
   - Estrategias: eliminar filas/columnas, imputar valores o crear indicadores de “dato faltante”.  

4. **Estadísticas descriptivas y univariadas**  
   - Media vs mediana (sesgo de la distribución).  
   - Desviación estándar (variabilidad, posibles outliers).  
   - Mínimos/máximos sospechosos.  
   - Histogramas para ver forma (normal, sesgada, bimodal, uniforme, picos extraños).  

5. **Análisis de variables categóricas**  
   - Distribución de categorías (ej. categorías de apps, tipo de app, content rating).  
   - Detección de clases dominantes o categorías poco representadas.  

6. **Correlaciones y relaciones entre variables**  
   - Matriz de correlación de Pearson para variables numéricas.  
   - Identificar relaciones fuertes, moderadas o débiles.  
   - Importante: recordar que **correlación ≠ causalidad**.  

---

**Nota:**  
No siempre es necesario aplicar todos los pasos con igual profundidad.  
- Para este proyecto, el foco está en **identificar variables relevantes para predecir el rating** y **limpiar datos inconsistentes**.  
- Otros análisis más complejos (ej. NLP sobre descripciones) se pueden dejar como trabajo futuro (según trabajos de referencia investigados).

---


### 2.2.1 Descarga de datos  

En este paso descargamos el dataset de Google Play Store desde Kaggle y lo organizamos en la estructura de carpetas del proyecto.  

1. Usamos la librería `kagglehub` para acceder al dataset público **`lava18/google-play-store-apps`** directamente desde Kaggle.  
2. Se define una ruta clara dentro del proyecto para almacenar los datos originales: `../data/original/google-play-store/`. Esto ayuda a mantener la reproducibilidad y una estructura organizada.  
3. Con la función `shutil.copytree` copiamos los archivos descargados a la carpeta destino. De esta forma, el dataset queda disponible en nuestro directorio de trabajo para su análisis posterior.  


In [ ]:
import kagglehub
import shutil

def download_data(origin_repository, target_folder):
    # Descargar dataset
    path = kagglehub.dataset_download(origin_repository)
    
    # Copiar los archivos descargados
    shutil.copytree(path, target_folder, dirs_exist_ok=True)
    
    print("Dataset guardado:", destino)

download_data("lava18/google-play-store-apps", "../data/original/google-play-store")

### 2.2.2 Carga de datos  

En este paso realizamos la **lectura del archivo CSV** que contiene el dataset descargado previamente.  

- Definimos una función `load_data(path, file)` que recibe la ruta y el nombre del archivo, y lo carga con `pandas.read_csv()`.  
- Cargamos el dataset principal en la variable `applications_data` desde la carpeta `../data/original/google-play-store/`.  
- Incluimos una verificación simple:  
  - Si el dataset se carga con éxito, se imprime `"Dataset loaded"`.  
  - En caso contrario, se muestra un mensaje de error.  

Con esta validación aseguramos que el archivo esté disponible y correctamente leído antes de continuar con el análisis exploratorio.


In [ ]:
import pandas as pd
import numpy as np

def load_data(path, file):
    return pd.read_csv(f"{path}/{file}")

def convert_numeric_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convierte a numéricas solo las columnas que deberían serlo, sin tocar 'Size'.
    Usa to_numeric(errors='coerce') para evitar ValueError si aparece texto.
    """
    df = df.copy()

    # Rating
    if "Rating" in df.columns:
        df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")

    # Reviews: quitar comas y cualquier carácter no numérico/punto
    if "Reviews" in df.columns:
        df["Reviews"] = (
            df["Reviews"].astype(str)
            .str.replace(r"[^\d.]", "", regex=True)
            .pipe(pd.to_numeric, errors="coerce")
        )

    # Installs: quitar +, comas y cualquier carácter no numérico/punto
    if "Installs" in df.columns:
        df["Installs Numeric"] = (
            df["Installs"].astype(str)
            .str.replace(r"[^\d.]", "", regex=True)
            .pipe(pd.to_numeric, errors="coerce")
        )

    # Price: quitar $ y cualquier carácter no numérico/punto
    if "Price" in df.columns:
        df["Price"] = (
            df["Price"].astype(str)
            .str.replace(r"[^\d.]", "", regex=True)
            .pipe(pd.to_numeric, errors="coerce")
        )


    if "Size" in df.columns:
        def parse_size(x):
            if isinstance(x, str):
                x = x.strip()
                if x.endswith("M"):
                    return float(x[:-1])
                elif x.endswith("k") or x.endswith("K"):
                    return float(x[:-1]) / 1024  # KB -> MB
                else:
                    return np.nan
            return np.nan
        df["Size"] = df["Size"].apply(parse_size)

    return df


temp_applications_data = load_data("../data/original/google-play-store", "googleplaystore.csv")
applications_data = convert_numeric_columns(temp_applications_data)


if len(applications_data):
    print("Dataset cargado")
else:
    print("Error cargando dataset")


### 2.2.3 Vista rápida del dataset

**Dimensiones y columnas**
- Registros: **10,841** filas.
- Columnas: actualmente **14**; **originalmente eran 13** y se **añadió** una columna derivada: **`Installs Numeric`** para análisis con describe.
- Memoria aproximada: **~1.2 MB**.

**Tipos de datos (y transformaciones realizadas)**
- Numéricas (`float64`): `Rating`, `Reviews`, `Size`, `Price`, **`Installs Numeric`**.
- Categóricas / texto (`object`): `App`, `Category`, `Installs` *(forma original con “1,000+”)*, `Type`, `Content Rating`, `Genres`, `Last Updated`, `Current Ver`, `Android Ver`.
- Transformaciones ya aplicadas:
  - **`Installs`** se **conservó** en su formato original (categórico con “+” y comas) **y** se creó **`Installs Numeric`** mapeando esos rangos a números (0 … 1,000,000,000).
  - **`Price`**, **`Reviews`** y **`Size`** fueron normalizadas/parseadas a **numérico** para análisis y modelado.

**Valores faltantes (no-null count → faltantes aprox.)**
- `Rating`: 9,367 → **1,474 faltantes (~13.6%)**.
- `Size`: 9,145 → **1,696 faltantes (~15.6%)**.
- `Current Ver`: 10,833 → **8 faltantes (~0.07%)**.
- `Android Ver`: 10,838 → **3 faltantes (~0.03%)**.
- `Content Rating`: 10,840 → **1 faltante (~0.01%)**.
- `Price`: 10,840 → **1 faltante (~0.01%)**.
- `Installs Numeric`: 10,840 → **1 faltante (~0.01%)**.
- Resto de columnas: **sin faltantes**.

**Duplicados:**
-   Se identificaron **483 filas duplicadas** (≈ **4.46%** del
    dataset).\
-   Ejemplos de duplicados incluyen apps como:
    -   *Quick PDF Scanner + OCR FREE*\
    -   *Box*\
    -   *Google My Business*\
    -   *ZOOM Cloud Meetings*\
    -   *join.me -- Simple Meetings*\

**Rangos y valores sospechosos (según `describe()`)**
- `Rating`: **min = 1.0**, **max = 19.0** → **19** es inválido para la escala 1–5 (error de dato a corregir).
- `Reviews`: media ~ **444k**, **p75 ≈ 54,768**, **max ≈ 78M** → valores altos plausibles; tratar como **outliers**.
- `Size` (MB): media ~ **21.5**, **p50 = 13**, **p75 = 30**, **max = 100** → distribución sesgada a la derecha; mínimos muy bajos (**0.01**) a revisar.
- `Price` (USD): **mediana = 0** y **p75 = 0** → la mayoría son **apps gratuitas**; **max = 400** sugiere outliers de precio.
- `Installs Numeric`: **p25 = 1,000**, **p50 = 100,000**, **p75 = 5,000,000**, **max = 1,000,000,000** → escala muy amplia; conviene usar **transformaciones log** o **binning** en el EDA/modelado.

**Conclusión inicial**
- Los **faltantes** más relevantes están en `Rating` y `Size`; habrá que decidir estategía para aumentar, imputar o nivelar los datos.
- Existen **outliers (no legítimos)** (ej. `Rating = 19`) y variables con **colas largas** (ej. `Reviews`, `Installs Numeric`, `Price`).
- Eliminar **duplicados** para evitar sesgos de análisis y que no introduzcan ruidos.


In [ ]:
print("=" * 80)
print("INFORMACIÓN GENERAL DEL DATASET".center(80))
print("=" * 80)

display(applications_data.head().style.background_gradient(cmap='RdYlGn', subset=['Rating']))

# Información detallada
print("\n" + "=" * 80)
print("ESTRUCTURA DE DATOS".center(80))
print("=" * 80)
applications_data.info()

# Estadísticas descriptivas
print("\n" + "=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS".center(80))
print("=" * 80)
display(applications_data.describe().round(2).T)

print("\n" + "=" * 80)
print("DATOS DUPLICADOS".center(80))
print("=" * 80)

# Contar duplicados
num_duplicados = applications_data.duplicated().sum()
print(f"Total de registros duplicados: {num_duplicados}")

# Mostrar ejemplos de duplicados si existen
if num_duplicados > 0:
    print("\nEjemplos de filas duplicadas:\n")
    display(applications_data[applications_data.duplicated()].head())
else:
    print("No se encontraron registros duplicados.")

print("=" * 80)
print("INFORMACIÓN GENERAL DEL DATASET".center(80))
print("=" * 80)

display(applications_data.head().style.background_gradient(cmap='RdYlGn', subset=['Rating']))
